In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib

# Preproccessing 


In [2]:
%cd C:\Users\user\Documents\Hackathon
%pwd

C:\Users\user\Documents\Hackathon


'C:\\Users\\user\\Documents\\Hackathon'

In [9]:
rel_col=['match_id','venue','innings','ball','batting_team','bowling_team','striker','bowler','runs_off_bat','extras']

In [10]:
#df = pd.read_csv("../Hackathon/Dataset/335982.csv",usecols=rel_col)

In [11]:
df = pd.read_csv("all_matches.csv",usecols=rel_col)

In [33]:
#df = pd.read_csv("../Hackathon/Modi.csv",usecols=rel_col)

In [253]:
#df = pd.read_csv("../Hackathon/Dataset/335982.csv")

In [6]:
df.shape

(200080, 10)

In [74]:
df.head()

,match_id,venue,innings,ball,batting_team,bowling_team,striker,bowler,runs_off_bat,extras
0,335982,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,0,1
1,335982,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0
2,335982,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,1
3,335982,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0
4,335982,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,0


In [12]:
df=df[df['ball']<=5.6]
df['total_runs']=df['runs_off_bat']+df['extras']
df=df.drop(columns=['runs_off_bat','extras'])
df=df[df['innings']<=2]
df=df[df['match_id']>=1216492]
#df=df[df['match_id']>=1243388]
#df=df.drop(columns=['match_id'])Feroz Shah Kotla
df['venue']=df['venue'].replace('Narendra Modi Stadium, Ahmedabad','Narendra Modi Stadium')
df['venue']=df['venue'].replace('Arun Jaitley Stadium, Delhi','Arun Jaitley Stadium')
df['venue']=df['venue'].replace('Wankhede Stadium, Mumbai','Wankhede Stadium')
df['venue']=df['venue'].replace( ['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium, Chepauk, Chennai'],'MA Chidambaram Stadium')
df['batting_team']=df['batting_team'].replace('Delhi Daredevils','Delhi Capitals')
df['bowling_team']=df['bowling_team'].replace('Delhi Daredevils','Delhi Capitals')
df['batting_team']=df['batting_team'].replace('Kings XI Punjab','Punjab Kings')
df['bowling_team']=df['bowling_team'].replace('Kings XI Punjab','Punjab Kings')

In [36]:
#df[df['batting_team']=='Chennai Super Kings'].tail(50)

In [76]:
df.head()

,match_id,venue,innings,ball,batting_team,bowling_team,striker,bowler,total_runs
179058,1216492,Sheikh Zayed Stadium,1,0.1,Mumbai Indians,Chennai Super Kings,RG Sharma,DL Chahar,4
179059,1216492,Sheikh Zayed Stadium,1,0.2,Mumbai Indians,Chennai Super Kings,RG Sharma,DL Chahar,1
179060,1216492,Sheikh Zayed Stadium,1,0.3,Mumbai Indians,Chennai Super Kings,Q de Kock,DL Chahar,2
179061,1216492,Sheikh Zayed Stadium,1,0.4,Mumbai Indians,Chennai Super Kings,Q de Kock,DL Chahar,4
179062,1216492,Sheikh Zayed Stadium,1,0.5,Mumbai Indians,Chennai Super Kings,Q de Kock,DL Chahar,0


# Calculate Wickets

In [13]:
ipl_data=df.groupby(['match_id','venue','innings','batting_team','bowling_team']).striker.nunique()-2

In [14]:
ipl_data.tail()

match_id  venue                  innings  batting_team                 bowling_team               
1254082   Narendra Modi Stadium  2        Delhi Capitals               Kolkata Knight Riders          0
1254083   Narendra Modi Stadium  1        Punjab Kings                 Royal Challengers Bangalore    1
                                 2        Royal Challengers Bangalore  Punjab Kings                   1
1254084   Arun Jaitley Stadium   1        Chennai Super Kings          Mumbai Indians                 1
                                 2        Mumbai Indians               Chennai Super Kings            0
Name: striker, dtype: int64

In [15]:
ipl_data=ipl_data.reset_index()
ipl_data=ipl_data.rename(columns={"striker":"Wicket"})

# Cacualting total runs

In [16]:
total=df.groupby(['match_id','venue','innings','batting_team','bowling_team']).total_runs.sum()
total=total.reset_index()
ipl_data['total_runs']=total['total_runs']


In [17]:
#ipl_data.head(172)
ipl_data.head()

,match_id,venue,innings,batting_team,bowling_team,Wicket,total_runs
0,1216492,Sheikh Zayed Stadium,1,Mumbai Indians,Chennai Super Kings,2,51
1,1216492,Sheikh Zayed Stadium,2,Chennai Super Kings,Mumbai Indians,2,31
2,1216493,Dubai International Cricket Stadium,1,Delhi Capitals,Punjab Kings,3,23
3,1216493,Dubai International Cricket Stadium,2,Punjab Kings,Delhi Capitals,3,35
4,1216494,Sheikh Zayed Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,4,17


# Recent venue score

In [18]:
venue=ipl_data.groupby(['venue','innings']).total_runs.mean().reset_index()
runs=[]
for data in venue.index:
    rate=ipl_data[ipl_data['venue'].isin([venue['venue'][data]]) & (ipl_data['innings']==venue['innings'][data])]
    rate=rate.tail(10)
    mean_total=rate.total_runs.mean()
    runs.append(mean_total)
venue['total_runs']=runs
#economy=economy.drop(columns=['total_runs'])
venue.head()

,venue,innings,total_runs
0,Arun Jaitley Stadium,1,45.000000
1,Arun Jaitley Stadium,2,52.333333
2,Dubai International Cricket Stadium,1,47.300000
3,Dubai International Cricket Stadium,2,44.400000
4,MA Chidambaram Stadium,1,43.700000


In [19]:
venue[venue['venue'].isin(['Arun Jaitley Stadium'])]

,venue,innings,total_runs
0,Arun Jaitley Stadium,1,45.000000
1,Arun Jaitley Stadium,2,52.333333


# Caculate  Strike rate

In [20]:
#df.groupby(['match_id','batting_team','striker']).ball.count()
#df.groupby(['match_id','batting_team','striker']).total_runs.sum()
#df.groupby(['match_id','bowling_team','bowler']).total_runs.sum()
#df1=df.groupby(['match_id','bowling_team','bowler']).ball.count().reset_index()
#df2=df.groupby(['match_id','bowling_team','bowler']).total_runs.sum().reset_index()
df1=pd.pivot_table(df, index=['match_id','innings','batting_team','striker'],values=['total_runs'],aggfunc=np.sum)
df2=pd.pivot_table(df, index=['match_id','innings','batting_team','striker'],values='ball',aggfunc=len)
strike_rate=pd.concat([df2,df1],axis=1)


In [21]:
strike_rate['str_rate']=strike_rate['total_runs']/strike_rate['ball']


In [22]:
strike_rate=strike_rate.groupby(['match_id','innings','batting_team']).str_rate.mean().reset_index()
strike_rate

,match_id,innings,batting_team,str_rate
0,1216492,1,Mumbai Indians,1.010227
1,1216492,2,Chennai Super Kings,0.778234
2,1216493,1,Delhi Capitals,0.555311
3,1216493,2,Punjab Kings,0.681579
4,1216494,1,Kolkata Knight Riders,0.404437
...,...,...,...,...
179,1254082,2,Delhi Capitals,1.869748
180,1254083,1,Punjab Kings,1.539855
181,1254083,2,Royal Challengers Bangalore,1.037037
182,1254084,1,Chennai Super Kings,1.260459


In [23]:
ipl_data['str_rate']=strike_rate['str_rate']


In [24]:
ipl_data.tail(20)

,match_id,venue,innings,batting_team,bowling_team,Wicket,total_runs,str_rate
164,1254075,Wankhede Stadium,1,Kolkata Knight Riders,Rajasthan Royals,0,25,0.657895
165,1254075,Wankhede Stadium,2,Rajasthan Royals,Kolkata Knight Riders,2,50,1.313113
166,1254076,Wankhede Stadium,1,Chennai Super Kings,Royal Challengers Bangalore,0,47,1.266082
167,1254076,Wankhede Stadium,2,Royal Challengers Bangalore,Chennai Super Kings,2,65,1.591912
168,1254077,MA Chidambaram Stadium,1,Delhi Capitals,Sunrisers Hyderabad,0,51,1.276398
169,1254077,MA Chidambaram Stadium,2,Sunrisers Hyderabad,Delhi Capitals,2,56,1.048611
170,1254078,Narendra Modi Stadium,1,Punjab Kings,Kolkata Knight Riders,0,37,1.004412
171,1254078,Narendra Modi Stadium,2,Kolkata Knight Riders,Punjab Kings,3,42,0.855769
172,1254079,Narendra Modi Stadium,1,Royal Challengers Bangalore,Delhi Capitals,2,36,0.838294
173,1254079,Narendra Modi Stadium,2,Delhi Capitals,Royal Challengers Bangalore,2,43,1.067857


# Calculate economy rate

In [25]:
df1=pd.pivot_table(df, index=['match_id','innings','bowling_team','bowler'],values=['total_runs'],aggfunc=np.sum)
df2=pd.pivot_table(df, index=['match_id','innings','bowling_team','bowler'],values='ball',aggfunc=len)
economy=pd.concat([df2,df1],axis=1)


In [26]:
df.groupby(['striker']).total_runs.sum().reset_index().set_index('striker')


,total_runs
striker,
AB de Villiers,35
AD Russell,14
AJ Finch,216
AM Rahane,92
AT Rayudu,54
...,...
V Shankar,3
Virat Singh,0
WP Saha,85


In [27]:
economy['ec_rate']=economy['total_runs']/economy['ball']
economy

ball  total_runs   ec_rate
match_id innings bowling_team        bowler                               
1216492  1       Chennai Super Kings DL Chahar  12.0          20  1.666667
                                     L Ngidi     8.0          18  2.250000
                                     PP Chawla   6.0           3  0.500000
                                     SM Curran  12.0          10  0.833333
         2       Mumbai Indians      JJ Bumrah   6.0           8  1.333333
...                                              ...         ...       ...
1254084  1       Mumbai Indians      JJ Bumrah   6.0           9  1.500000
                                     TA Boult   18.0          23  1.277778
         2       Chennai Super Kings DL Chahar  19.0          27  1.421053
                                     L Ngidi     6.0          14  2.333333
                                     SM Curran  13.0          17  1.307692

[665 rows x 3 columns]

In [28]:
economy=economy.groupby(['match_id','innings','bowling_team']).ec_rate.mean().reset_index()
economy

,match_id,innings,bowling_team,ec_rate
0,1216492,1,Chennai Super Kings,1.312500
1,1216492,2,Mumbai Indians,0.927350
2,1216493,1,Punjab Kings,0.714286
3,1216493,2,Delhi Capitals,0.822802
4,1216494,1,Royal Challengers Bangalore,0.713675
...,...,...,...,...
179,1254082,2,Kolkata Knight Riders,1.783333
180,1254083,1,Royal Challengers Bangalore,1.273504
181,1254083,2,Punjab Kings,0.907407
182,1254084,1,Mumbai Indians,1.398148


In [29]:
ipl_data['ec_rate']=economy['ec_rate']

In [30]:
ipl_data.tail()

,match_id,venue,innings,batting_team,bowling_team,Wicket,total_runs,str_rate,ec_rate
179,1254082,Narendra Modi Stadium,2,Delhi Capitals,Kolkata Knight Riders,0,67,1.869748,1.783333
180,1254083,Narendra Modi Stadium,1,Punjab Kings,Royal Challengers Bangalore,1,49,1.539855,1.273504
181,1254083,Narendra Modi Stadium,2,Royal Challengers Bangalore,Punjab Kings,1,36,1.037037,0.907407
182,1254084,Arun Jaitley Stadium,1,Chennai Super Kings,Mumbai Indians,1,49,1.260459,1.398148
183,1254084,Arun Jaitley Stadium,2,Mumbai Indians,Chennai Super Kings,0,58,1.522222,1.687359


In [31]:
ipl_data=ipl_data.drop(columns=['match_id'])
ipl_data.to_csv("Data.csv",index='False')

# Create MATCH DATA

In [84]:

Match27=ipl_data[((ipl_data['bowling_team'].isin(['Chennai Super Kings'])  | ipl_data['bowling_team'].isin(['Mumbai Indians'])| ipl_data['batting_team'].isin(['Chennai Super Kings'])  | ipl_data['batting_team'].isin(['Mumbai Indians'])) & (ipl_data['match_id']>=1216492))|(ipl_data['venue'].isin(['Arun Jaitley Stadium']))]
#Match23=ipl_data[((ipl_data['bowling_team'].isin(['Chennai Super Kings'])  | ipl_data['bowling_team'].isin(['Sunrisers Hyderabad'])| ipl_data['batting_team'].isin(['Chennai Super Kings'])  | ipl_data['batting_team'].isin(['Sunrisers Hyderabad'])) )|(ipl_data['venue'].isin(['Arun Jaitley Stadium']))]
Match27=Match27.drop(columns=['match_id'])
#Match21=Match21[Match21['total_runs']>=30]
Match27=Match27.reset_index(drop=True)
Match27

,venue,innings,batting_team,bowling_team,Wicket,total_runs,str_rate,ec_rate
0,Arun Jaitley Stadium,1,Punjab Kings,Delhi Capitals,2,51,1.311218,1.342593
1,Arun Jaitley Stadium,2,Delhi Capitals,Punjab Kings,3,48,1.880000,1.634615
2,Arun Jaitley Stadium,1,Delhi Capitals,Kolkata Knight Riders,0,57,1.661905,1.550000
3,Arun Jaitley Stadium,2,Kolkata Knight Riders,Delhi Capitals,4,51,1.203704,1.479167
4,Arun Jaitley Stadium,1,Delhi Capitals,Rajasthan Royals,1,62,1.166667,1.783333
...,...,...,...,...,...,...,...,...
103,Wankhede Stadium,2,Royal Challengers Bangalore,Chennai Super Kings,2,65,1.591912,1.645363
104,Arun Jaitley Stadium,1,Sunrisers Hyderabad,Chennai Super Kings,1,39,1.152778,1.026316
105,Arun Jaitley Stadium,2,Chennai Super Kings,Sunrisers Hyderabad,0,50,1.342105,1.538012
106,Arun Jaitley Stadium,1,Rajasthan Royals,Mumbai Indians,0,47,1.449091,1.541667


In [58]:
Match26=ipl_data[((ipl_data['bowling_team'].isin(['Royal Challengers Bangalore'])  | ipl_data['bowling_team'].isin(['Punjab Kings'])| ipl_data['batting_team'].isin(['Royal Challengers Bangalore'])  | ipl_data['batting_team'].isin(['Punjab Kings'])) & (ipl_data['match_id']>=1216492))|(ipl_data['venue'].isin(['Narendra Modi Stadium']))]
#Match23=ipl_data[((ipl_data['bowling_team'].isin(['Chennai Super Kings'])  | ipl_data['bowling_team'].isin(['Sunrisers Hyderabad'])| ipl_data['batting_team'].isin(['Chennai Super Kings'])  | ipl_data['batting_team'].isin(['Sunrisers Hyderabad'])) )|(ipl_data['venue'].isin(['Arun Jaitley Stadium']))]
Match26=Match26.drop(columns=['match_id'])
#Match21=Match21[Match21['total_runs']>=30]
Match26=Match26.reset_index(drop=True)


In [59]:
Match26[Match26['venue']=='Narendra Modi Stadium'].shape
#Match23.shape

(16, 8)

In [60]:
a=Match25[Match25['venue']=='Narendra Modi Stadium']
a
#a.groupby(['venue','innings']).total_runs.mean()

NameError: name 'Match25' is not defined

In [ ]:
Match25[Match25['venue']=='Narendra Modi Stadium']

In [97]:
#ipl_data=ipl_data.drop(columns=['match_id'])
#ipl_data=ipl_data.drop(columns=['balls'])

In [85]:
Match27.iloc[0:50,:]

,venue,innings,batting_team,bowling_team,Wicket,total_runs,str_rate,ec_rate
0,Arun Jaitley Stadium,1,Punjab Kings,Delhi Capitals,2,51,1.311218,1.342593
1,Arun Jaitley Stadium,2,Delhi Capitals,Punjab Kings,3,48,1.880000,1.634615
2,Arun Jaitley Stadium,1,Delhi Capitals,Kolkata Knight Riders,0,57,1.661905,1.550000
3,Arun Jaitley Stadium,2,Kolkata Knight Riders,Delhi Capitals,4,51,1.203704,1.479167
4,Arun Jaitley Stadium,1,Delhi Capitals,Rajasthan Royals,1,62,1.166667,1.783333
5,Arun Jaitley Stadium,2,Rajasthan Royals,Delhi Capitals,0,79,1.818333,1.995238
6,Arun Jaitley Stadium,1,Delhi Capitals,Sunrisers Hyderabad,2,38,1.056743,1.208333
7,Arun Jaitley Stadium,2,Sunrisers Hyderabad,Delhi Capitals,1,51,1.438889,1.333333
8,Arun Jaitley Stadium,1,Delhi Capitals,Royal Challengers Bangalore,2,44,1.046154,1.149573
9,Arun Jaitley Stadium,2,Royal Challengers Bangalore,Delhi Capitals,2,58,1.222222,1.669643


In [86]:

Match27.to_csv("Match27.csv",index='False')

In [170]:

Match25.to_csv("Match25.csv",index='False')

In [87]:
Match27[['total_runs']].mean()

total_runs    46.472222
dtype: float64

In [88]:
Match25[['total_runs']].mean()

NameError: name 'Match25' is not defined

In [42]:
#sns.pairplot(Match21,hue='innings')

In [43]:
#Match21.describe().transpose()

In [89]:
Match27.tail()

,venue,innings,batting_team,bowling_team,Wicket,total_runs,str_rate,ec_rate
103,Wankhede Stadium,2,Royal Challengers Bangalore,Chennai Super Kings,2,65,1.591912,1.645363
104,Arun Jaitley Stadium,1,Sunrisers Hyderabad,Chennai Super Kings,1,39,1.152778,1.026316
105,Arun Jaitley Stadium,2,Chennai Super Kings,Sunrisers Hyderabad,0,50,1.342105,1.538012
106,Arun Jaitley Stadium,1,Rajasthan Royals,Mumbai Indians,0,47,1.449091,1.541667
107,Arun Jaitley Stadium,2,Mumbai Indians,Rajasthan Royals,0,49,1.335913,1.562500


# Calculate economy rate of Players


economy=df.groupby(['bowler']).ball.count()
runs=df.groupby(['bowler']).total_runs.sum()
#economy=df[df['bowler'].isin(['Abhishek Sharma'])]

runs=runs.reset_index()

economy=economy.reset_index()
economy['total_runs']=runs['total_runs']
economy['ec_rate']=economy['total_runs']/economy['ball']



In [32]:
economy=df.groupby(['bowler']).total_runs.sum().reset_index().set_index('bowler')
ec_rate=[]
for data in economy.index:
    rate=df[df['bowler'].isin([data])]
    rate=rate.tail(120)
    economy_rate=(rate.total_runs.sum()/rate.ball.count())
    ec_rate.append(economy_rate)
economy['ec_rate']=ec_rate
economy=economy.drop(columns=['total_runs'])

In [33]:
economy.to_csv("economy_rate.csv",index='False')

In [34]:
economy

,ec_rate
bowler,
A Mishra,1.666667
A Nortje,1.433333
AD Russell,1.222222
AF Milne,2.714286
AJ Tye,1.166667
...,...
UT Yadav,1.500000
V Shankar,1.272727
VR Aaron,1.744681


In [35]:
economy.loc[['DR Sams','Mohammed Siraj','KA Jamieson']].ec_rate.mean()*36
#economy.loc[[]]

42.2495126705653

In [36]:
Mohammed Shami,MC Henriques,DJ Hooda



SyntaxError: invalid syntax (<ipython-input-36-2e031f4c1f3e>, line 1)

In [37]:
economy.loc[['Mohammed Siraj','KA Jamieson','KW Richardson','DR Sams','YS Chahal','NA Saini'],['ec_rate']].ec_rate.mean()*36 
economy.loc[['Mohammed Siraj','KA Jamieson','KW Richardson','DR Sams','YS Chahal','NA Saini'],['ec_rate']] 

,ec_rate
bowler,
Mohammed Siraj,1.166667
KA Jamieson,1.283951
KW Richardson,1.666667
DR Sams,1.070175
YS Chahal,1.636364
NA Saini,1.136752


# Calculate Strike rate of Players

In [38]:
#df = pd.read_csv("all_matches.csv",usecols=rel_col)
#df=df[df['ball']<=5.6]
#df['total_runs']=df['runs_off_bat']+df['extras']
#df=df.drop(columns=['runs_off_bat','extras'])
#df=df[df['innings']<=2]

In [39]:
a=df.groupby(['striker']).total_runs.sum().reset_index().set_index('striker')
str_rate=[]
for data in a.index:
    rate=df[df['striker'].isin([data])]
    rate=rate.tail(120)
    strike_rate=(rate.total_runs.sum()/rate.ball.count())
    str_rate.append(strike_rate)
a['str_rate']=str_rate
    #print(str_rate)

In [40]:
a=a.drop(columns=['total_runs'])
   

In [41]:
a.to_csv("strike_rate.csv",index='False')

In [42]:
strike_rate=pd.read_csv('strike_rate.csv')
strike_rate=strike_rate.set_index('striker')
strike_rate

,str_rate
striker,
AB de Villiers,1.093750
AD Russell,3.500000
AJ Finch,1.141667
AM Rahane,1.045455
AT Rayudu,1.058824
...,...
V Shankar,0.428571
Virat Singh,0.000000
WP Saha,1.440678


In [104]:
#strike_rate=strike_rate.drop(['Unnamed: 0'],axis=1)
#strike_rate

In [105]:
KL Rahul,MA Agarwal,CH Gayle



SyntaxError: invalid syntax (<ipython-input-105-ef0bcd78bb64>, line 1)

In [43]:
strike_rate.loc[['KL Rahul','P Simran Singh','CH Gayle']].str_rate.mean()*36
#strike_rate.loc[['KL Rahul','P Simran Singh','CH Gayle']]

42.96292134831461

In [44]:
strike_rate.loc[['D Padikkal','V Kohli','RM Patidar']].str_rate.mean()*36
#strike_rate.loc[['D Padikkal','V Kohli','GJ Maxwell','RM Patidar','Washington Sundar']]

40.44

In [91]:
D Padikkal,V Kohli,Washington Sundar


SyntaxError: invalid syntax (<ipython-input-91-69ddc21bc3bc>, line 1)

# Trainer


In [45]:
data = pd.read_csv("../Hackathon/Data.csv")
data=data.drop(['Unnamed: 0'],axis=1)

In [105]:
data = pd.read_csv("../Hackathon/Match27.csv")
data=data.drop(['Unnamed: 0'],axis=1)

In [46]:
data

,venue,innings,batting_team,bowling_team,Wicket,total_runs,str_rate,ec_rate
0,Sheikh Zayed Stadium,1,Mumbai Indians,Chennai Super Kings,2,51,1.010227,1.312500
1,Sheikh Zayed Stadium,2,Chennai Super Kings,Mumbai Indians,2,31,0.778234,0.927350
2,Dubai International Cricket Stadium,1,Delhi Capitals,Punjab Kings,3,23,0.555311,0.714286
3,Dubai International Cricket Stadium,2,Punjab Kings,Delhi Capitals,3,35,0.681579,0.822802
4,Sheikh Zayed Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,4,17,0.404437,0.713675
...,...,...,...,...,...,...,...,...
179,Narendra Modi Stadium,2,Delhi Capitals,Kolkata Knight Riders,0,67,1.869748,1.783333
180,Narendra Modi Stadium,1,Punjab Kings,Royal Challengers Bangalore,1,49,1.539855,1.273504
181,Narendra Modi Stadium,2,Royal Challengers Bangalore,Punjab Kings,1,36,1.037037,0.907407
182,Arun Jaitley Stadium,1,Chennai Super Kings,Mumbai Indians,1,49,1.260459,1.398148


In [47]:
ve=LabelEncoder() #Venue encoder
te=LabelEncoder() #Team encoder

In [48]:
#te.fit_transform(ipl_data['batting_team'])
te.fit_transform(data['batting_team'])
data['venue']=ve.fit_transform(data['venue'])
data['batting_team']=te.transform(data['batting_team'])
data['bowling_team']=te.transform(data['bowling_team'])

#data=data[['venue','innings','batting_team','bowling_team','Wicket','str_rate','economy','total_runs']]

In [49]:
data.iloc[:,0:6].to_numpy()

array([[ 5,  1,  5,  0,  2, 51],
       [ 5,  2,  0,  5,  2, 31],
       [ 1,  1,  1,  6,  3, 23],
       ...,
       [ 3,  2,  8,  6,  1, 36],
       [ 0,  1,  0,  5,  1, 49],
       [ 0,  2,  5,  0,  0, 58]], dtype=int64)

In [50]:
list(te.classes_)
list(ve.classes_)

['Arun Jaitley Stadium',
 'Dubai International Cricket Stadium',
 'MA Chidambaram Stadium',
 'Narendra Modi Stadium',
 'Sharjah Cricket Stadium',
 'Sheikh Zayed Stadium',
 'Wankhede Stadium']

In [51]:
list(te.classes_)

['Chennai Super Kings',
 'Delhi Capitals',
 'England',
 'India',
 'Kolkata Knight Riders',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad']

In [52]:
te.transform(["Sunrisers Hyderabad"])

array([9], dtype=int64)

In [53]:
ve.transform(["Wankhede Stadium"])

array([6], dtype=int64)

In [54]:
list(te.inverse_transform([1, 3]))

['Delhi Capitals', 'India']

In [55]:
#get data into a numpy array
int_array=data.iloc[:,0:6].to_numpy()
flt_array=data.to_numpy()

In [56]:
int_array

array([[ 5,  1,  5,  0,  2, 51],
       [ 5,  2,  0,  5,  2, 31],
       [ 1,  1,  1,  6,  3, 23],
       ...,
       [ 3,  2,  8,  6,  1, 36],
       [ 0,  1,  0,  5,  1, 49],
       [ 0,  2,  5,  0,  0, 58]], dtype=int64)

In [57]:
flt_array[:,7]

array([1.3125    , 0.92735043, 0.71428571, 0.8228022 , 0.71367521,
       1.17566352, 1.33333333, 1.64583333, 1.45441595, 1.66185897,
       1.02083333, 0.88888889, 1.37962963, 1.37614871, 1.47916667,
       1.0625    , 1.25160256, 1.01190476, 1.18838764, 1.46783626,
       0.70655271, 1.58854167, 1.19987469, 1.05357143, 0.98878205,
       1.18253968, 1.06730769, 1.5       , 1.5       , 1.63461538,
       0.8125    , 1.        , 1.64308608, 0.875     , 1.1474359 ,
       1.24074074, 1.33333333, 1.11666667, 1.45128205, 0.92592593,
       1.33333333, 1.60416667, 1.33333333, 1.74404762, 0.83333333,
       1.58333333, 1.66666667, 1.6       , 1.20040486, 0.93650794,
       1.16452991, 0.98464912, 1.13960114, 1.48099817, 2.15277778,
       1.14029304, 1.46153846, 0.87037037, 0.67989418, 1.30555556,
       1.41586538, 1.29716117, 0.62962963, 1.4537037 , 2.0625    ,
       1.43650794, 1.11988304, 0.76442308, 0.82307692, 1.41666667,
       1.57051282, 1.76593407, 1.20655271, 1.05      , 1.09523

In [58]:
#anArray=np.delete(anArray, 2, 1)

In [59]:
X,y=int_array[:,:5],int_array[:,5]

In [60]:
X1,y1=flt_array[:,[1,4,6,7]],flt_array[:,5]

In [61]:
X1[:,:4]


array([[1.        , 2.        , 1.01022727, 1.3125    ],
       [2.        , 2.        , 0.77823427, 0.92735043],
       [1.        , 3.        , 0.55531136, 0.71428571],
       [2.        , 3.        , 0.68157895, 0.8228022 ],
       [1.        , 4.        , 0.40443723, 0.71367521],
       [2.        , 0.        , 1.20454545, 1.17566352],
       [1.        , 2.        , 1.01945971, 1.33333333],
       [2.        , 0.        , 1.49852941, 1.64583333],
       [1.        , 1.        , 1.42105263, 1.45441595],
       [2.        , 0.        , 1.41323529, 1.66185897],
       [1.        , 2.        , 0.90922619, 1.02083333],
       [2.        , 2.        , 0.71666667, 0.88888889],
       [1.        , 1.        , 1.41652661, 1.37962963],
       [2.        , 0.        , 1.36842105, 1.37614871],
       [1.        , 0.        , 1.51083591, 1.47916667],
       [2.        , 1.        , 1.21261487, 1.0625    ],
       [1.        , 2.        , 1.20328947, 1.25160256],
       [2.        , 1.        ,

### Predict for specific Match Data

In [121]:
X=np.concatenate((np.eye(6)[int_array[:,0]],
                  np.eye(2)[int_array[:,1]-1],
                  np.eye(10)[int_array[:,2]],
                  np.eye(10)[int_array[:,3]],
                  int_array[:,4].reshape(int_array[:,4].size,1)),axis=1)
X1=np.concatenate((np.eye(2)[int_array[:,1]-1],
                  X1[:,1:4]),axis=1)


### Predict for Overall Match Data

In [148]:

X=np.concatenate((np.eye(41)[int_array[:,0]],
                  np.eye(2)[int_array[:,1]-1],
                  np.eye(16)[int_array[:,2]],
                  np.eye(16)[int_array[:,3]],
                  int_array[:,4].reshape(int_array[:,4].size,1)),axis=1)
X1=np.concatenate((np.eye(2)[int_array[:,1]-1],
                  X1[:,1:4]),axis=1)

### Predict from last 2 IPL data


In [62]:
X=np.concatenate((np.eye(7)[int_array[:,0]],
                  np.eye(2)[int_array[:,1]-1],
                  np.eye(10)[int_array[:,2]],
                  np.eye(10)[int_array[:,3]],
                  int_array[:,4].reshape(int_array[:,4].size,1)),axis=1)
X1=np.concatenate((np.eye(2)[int_array[:,1]-1],
                  X1[:,1:4]),axis=1)

In [63]:
X

array([[0., 0., 0., ..., 0., 0., 2.],
       [0., 0., 0., ..., 0., 0., 2.],
       [0., 1., 0., ..., 0., 0., 3.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [64]:
X1.shape

(184, 5)

In [65]:
y.shape

(184,)

In [66]:
#pd.get_dummies(data,columns=['venue','batting_team','bowling_team'],prefix='')
#data=pd.concat([Chennai.get_dummies(Chennai['venue']),pd.get_dummies(data['innings'],prefix='innings'),pd.get_dummies(data['batting_team']),pd.get_dummies(data['bowling_team']),data['total_runs']], axis=1)

In [67]:
#data.iloc[:,0:81]

In [68]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05,random_state=10)
X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y1,test_size=0.05,random_state=10)

In [69]:
X1_test

array([[0.        , 1.        , 1.        , 1.24400871, 1.29716117],
       [1.        , 0.        , 2.        , 0.72470238, 0.99002849],
       [0.        , 1.        , 1.        , 1.22348485, 1.625     ],
       [0.        , 1.        , 2.        , 1.38303571, 1.25      ],
       [1.        , 0.        , 1.        , 1.26045884, 1.39814815],
       [0.        , 1.        , 0.        , 1.30625   , 1.4537037 ],
       [1.        , 0.        , 2.        , 0.90922619, 1.02083333],
       [0.        , 1.        , 1.        , 1.93269231, 1.95833333],
       [1.        , 0.        , 3.        , 0.55531136, 0.71428571],
       [0.        , 1.        , 2.        , 1.13229167, 1.14029304]])

In [70]:
y1_test

array([47., 32., 48., 45., 49., 47., 36., 66., 23., 43.])

Linear Regression

In [71]:
#fit a linear regressor
linearRegressor=LinearRegression()
linearRegressor1=LinearRegression()

In [72]:
check=linearRegressor.fit(X_train, y_train).predict(X_test)
check

array([49.17784719, 40.89713313, 52.73792787, 43.89387558, 35.85362753,
       51.46422549, 40.52301569, 51.97410695, 40.2171189 , 44.43337222])

In [73]:
X_train

array([[0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [74]:
from sklearn.metrics import r2_score
score =r2_score(y_test, check)
score

0.31566842310181287

In [75]:
linearRegressor.fit(X_train, y_train)
linearRegressor1.fit(X1_train, y1_train)

LinearRegression()

In [76]:
linearRegressor.score(X_test,y_test)
linearRegressor1.score(X1_test,y1_test)

0.8887225740268169

In [77]:
joblib.dump(linearRegressor, 'regression_model.joblib')
joblib.dump(linearRegressor1, 'regression_model1.joblib')
joblib.dump(ve, 'venue_encoder.joblib')
joblib.dump(te, 'team_encoder.joblib')

['team_encoder.joblib']

RANDOM FOREST


In [187]:
from sklearn.ensemble import RandomForestRegressor

In [188]:
random_regressor=RandomForestRegressor(n_estimators=10,random_state=40)
random_regressor.fit(X1_train, y1_train)

RandomForestRegressor(n_estimators=10, random_state=40)

In [189]:
random_regressor.score(X1_test,y1_test)

0.8989684669573969

In [190]:
y1_test

array([47., 32., 48., 45., 49., 47., 36., 66., 23., 43.])

In [191]:
check=random_regressor.fit(X1_train, y1_train).predict(X1_test)
check

array([46.7, 36.1, 52.7, 48.6, 48.8, 52.8, 41.5, 65.3, 24.1, 41.3])

In [192]:
joblib.dump(random_regressor, 'regression_model1.joblib')

['regression_model1.joblib']

# Predictor

In [93]:
%load predictor.py
..\Submission Format\predictor.py

SyntaxError: invalid syntax (<ipython-input-93-2f5f24b61d52>, line 2)

In [153]:
%pwd
%cd C:\Users\user\Documents\Hackathon

C:\Users\user\Documents\Hackathon


In [170]:
prediction = 0

In [194]:
with open('regression_model.joblib','rb') as f:
    regressor=joblib.load(f)
with open('regression_model1.joblib','rb') as f:
    regressor1=joblib.load(f)
with open('venue_encoder.joblib','rb') as f:
    venue_encoder=joblib.load(f)
with open('team_encoder.joblib','rb') as f:
    team_encoder=joblib.load(f) 

In [195]:
pwd

'C:\\Users\\user\\Documents\\Hackathon'

In [198]:
test_case=pd.read_csv('21_inn1.csv')

In [199]:
test_case['Wicket']=test_case['batsmen'].str.count(',')-1

In [200]:
test_case

,venue,innings,batting_team,bowling_team,batsmen,bowlers,Wicket
0,Narendra Modi Stadium,1,Punjab Kings,Kolkata Knight Riders,"MA Agarwal,KL Rahul,CH Gayle","Shivam Mavi,PJ Cummins,SP Narine",1


In [201]:
batsmen=test_case['batsmen'].str.split(pat=',',expand=True)
batsmen

,0,1,2
0,MA Agarwal,KL Rahul,CH Gayle


In [202]:
#batsmen=test_case['batsmen'].str.split(pat=',')
#batsmen=np.array(batsmen)

batsmen=test_case.values[0,4]
test_case.dtypes

venue           object
innings          int64
batting_team    object
bowling_team    object
batsmen         object
bowlers         object
Wicket           int64
dtype: object

In [203]:
str_rate=pd.read_csv('strike_rate.csv')
str_rate=str_rate.set_index('striker')
str_rate.loc[['F du Plessis','RD Gaikwad']]


,str_rate
striker,
F du Plessis,1.600000
RD Gaikwad,0.966667


In [204]:
batsmen=test_case['batsmen'].str.split(pat=',',expand=True)
n=0
rate=0
for i in range(len(batsmen.columns)):
    if batsmen.iloc[0,i] in str_rate.index:
        n=n+1
        rate=rate+str_rate.loc[batsmen.iloc[0,i],'str_rate']
rate=rate/n 
if(n>0):
    test_case['str_rate']=rate
else:
    test_case['str_rate']=str_rate.str_rate.mean() 

test_case    

,venue,innings,batting_team,bowling_team,batsmen,bowlers,Wicket,str_rate
0,Narendra Modi Stadium,1,Punjab Kings,Kolkata Knight Riders,"MA Agarwal,KL Rahul,CH Gayle","Shivam Mavi,PJ Cummins,SP Narine",1,1.232303


In [188]:
1.36*36


48.96

In [189]:
n

2

In [205]:
economy=pd.read_csv('economy_rate.csv')
economy

,bowler,ec_rate
0,A Mishra,1.666667
1,A Nortje,1.433333
2,AD Russell,1.222222
3,AF Milne,2.714286
4,AJ Tye,1.166667
...,...,...
94,UT Yadav,1.500000
95,V Shankar,1.272727
96,VR Aaron,1.744681
97,Washington Sundar,1.333333


In [206]:
economy=economy.set_index('bowler')
economy

,ec_rate
bowler,
A Mishra,1.666667
A Nortje,1.433333
AD Russell,1.222222
AF Milne,2.714286
AJ Tye,1.166667
...,...
UT Yadav,1.500000
V Shankar,1.272727
VR Aaron,1.744681


In [192]:
economy.loc[['DL Chahar','SM Curran','L Ngidi'], 'ec_rate']

bowler
DL Chahar    1.266667
SM Curran    1.091667
L Ngidi      1.538462
Name: ec_rate, dtype: float64

In [207]:
bowler=test_case['bowlers'].str.split(pat=',',expand=True)
n=0
rate=0
for i in range(len(bowler.columns)):
    
    if bowler.iloc[0,i] in economy.index:
        n=n+1
        rate=rate+economy.loc[bowler.iloc[0,i],'ec_rate']
if(n>0):
    rate=rate/n
    test_case['ec_rate']=rate
else:
    test_case['ec_rate']=economy.ec_rate.mean()
test_case    

,venue,innings,batting_team,bowling_team,batsmen,bowlers,Wicket,str_rate,ec_rate
0,Narendra Modi Stadium,1,Punjab Kings,Kolkata Knight Riders,"MA Agarwal,KL Rahul,CH Gayle","Shivam Mavi,PJ Cummins,SP Narine",1,1.232303,1.360428


In [194]:
1.2989*36
#1.188889*36

46.7604

In [208]:
n

3

In [196]:
if bowler.iloc[0,1] in economy.index:
    print(1)

1


In [209]:
test_case['venue']=venue_encoder.transform(test_case['venue'])
test_case['bowling_team']=team_encoder.transform(test_case['bowling_team'])
test_case['batting_team']=team_encoder.transform(test_case['batting_team'])

In [210]:
test_case=test_case[['venue','innings','batting_team','bowling_team','Wicket','ec_rate','str_rate']]

### Predict case for Specific Match

In [39]:
testArray=test_case.iloc[:,:5].to_numpy()
rateArray=test_case.iloc[:,4:7].to_numpy()
test_case=np.concatenate((np.eye(6)[testArray[:,0]],
                          np.eye(2)[testArray[:,1]-1],
                          np.eye(10)[testArray[:,2]],
                          np.eye(10)[testArray[:,3]],
                          testArray[:,4].reshape(testArray[:,4].size,1)),axis=1)
rate_case=np.concatenate((np.eye(2)[testArray[:,1]-1],
                  rateArray[:,:]),axis=1)
test_case

array([[1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]])

### Predict case for All Match

In [217]:
testArray=test_case.iloc[:,:5].to_numpy()
rateArray=test_case.iloc[:,4:7].to_numpy()
test_case=np.concatenate((np.eye(41)[testArray[:,0]],
                          np.eye(2)[testArray[:,1]-1],
                          np.eye(16)[testArray[:,2]],
                          np.eye(16)[testArray[:,3]],
                          testArray[:,4].reshape(testArray[:,4].size,1)),axis=1)
rate_case=np.concatenate((np.eye(2)[testArray[:,1]-1],
                  rateArray[:,:]),axis=1)
test_case

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]])

### Predict case for last two IPL edition


In [211]:
testArray=test_case.iloc[:,:5].to_numpy()
rateArray=test_case.iloc[:,4:7].to_numpy()
test_case=np.concatenate((np.eye(7)[testArray[:,0]],
                          np.eye(2)[testArray[:,1]-1],
                          np.eye(10)[testArray[:,2]],
                          np.eye(10)[testArray[:,3]],
                          testArray[:,4].reshape(testArray[:,4].size,1)),axis=1)
rate_case=np.concatenate((np.eye(2)[testArray[:,1]-1],
                  rateArray[:,:]),axis=1)
test_case

array([[0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]])

In [212]:
rate_case

array([[1.        , 0.        , 1.        , 1.36042805, 1.23230337]])

In [213]:
#testArray=test_case.iloc[:,:5].to_numpy()
#rateArray=test_case.iloc[:,4:7].to_numpy()
#test_case=np.concatenate((np.eye(6)[testArray[:,0]],np.eye(2)[testArray[:,1]-1],np.eye(8)[testArray[:,2]],np.eye(8)[testArray[:,3]],testArray[:,4].reshape(testArray[:,4].size,1)),axis=1)
int_prediction=regressor.predict(test_case)
flt_prediction=regressor1.predict(rate_case)

In [214]:
flt_prediction


array([48.2])

In [215]:
int_prediction

array([42.22015904])

In [216]:

prediction=(flt_prediction+int_prediction)/2

In [217]:
int(prediction)

45

In [206]:
test_case.shape

(1, 30)

In [207]:
regressor.predict(test_case)

array([47.72461713])

# Running Test

In [1]:
%pwd
%cd C:\Users\user\Documents\Hackathon
%pwd

C:\Users\user\Documents\Hackathon


'C:\\Users\\user\\Documents\\Hackathon'

In [28]:
%pwd
%cd C:\Users\user\Documents\Hackathon\Submission format - Copy\Match29
%pwd

C:\Users\user\Documents\Hackathon\Submission format - Copy\Match29


'C:\\Users\\user\\Documents\\Hackathon\\Submission format - Copy\\Match29'

In [9]:
%pwd
%cd C:\Users\user\Documents\Hackathon\Submission format - Copy
%pwd

C:\Users\user\Documents\Hackathon\Submission format - Copy


'C:\\Users\\user\\Documents\\Hackathon\\Submission format - Copy'

In [8]:
# %load predictor.py
### Custom definitions and classes if any ###
import pandas as pd
import numpy as np
import joblib

def predictRuns(testInput):
    prediction = 0
    ### Your Code Here ###
    
    with open('regression_model.joblib','rb') as f:
        regressor=joblib.load(f)
    with open('regression_model1.joblib','rb') as f:
        regressor1=joblib.load(f)
    with open('venue_encoder.joblib','rb') as f:
        venue_encoder=joblib.load(f)
    with open('team_encoder.joblib','rb') as f:
        team_encoder=joblib.load(f) 
        
    #read test data
    test_case=pd.read_csv(testInput)
    
    test_case['Wicket']=test_case['batsmen'].str.count(',')-1
    test_case['venue']=venue_encoder.transform(test_case['venue'])
    test_case['bowling_team']=team_encoder.transform(test_case['bowling_team'])
    test_case['batting_team']=team_encoder.transform(test_case['batting_team'])
    
    #Load Strike rate table
    str_rate=pd.read_csv('strike_rate.csv')
    str_rate=str_rate.set_index('Name')
    
    #Appending averaage batsmen strike rate
    batsmen=test_case['batsmen'].str.split(pat=',',expand=True)
    n=0
    rate=0
    for i in range(len(batsmen.columns)):
        if batsmen.iloc[0,i] in str_rate.index:
            n=n+1
            rate=rate+str_rate.loc[batsmen.iloc[0,i],'str_rate'] 
    if(n>0):
        rate=rate/n 
        test_case['str_rate']=rate
    else:
        test_case['str_rate']=str_rate.str_rate.mean() 
        
    #Load Economy rate table
    economy=pd.read_csv('economy_rate.csv')
    economy=economy.set_index('bowler')
    
    #Append average economy of bowlers
    bowler=test_case['bowlers'].str.split(pat=',',expand=True)
    n=0
    rate=0
    for i in range(len(bowler.columns)):
        if bowler.iloc[0,i] in economy.index:
            n=n+1
            rate=rate+economy.loc[bowler.iloc[0,i],'ec_rate']
    if(n>0):
        rate=rate/n
        test_case['ec_rate']=rate
    else:
        test_case['ec_rate']=economy.ec_rate.mean()
    
    test_case=test_case[['venue','innings','batting_team','bowling_team','Wicket','ec_rate','str_rate']]
    
    
    testArray=test_case.iloc[:,:5].to_numpy()
    rateArray=test_case.iloc[:,4:7].to_numpy()
    
    #Prediction with label encoders
    test_case=np.concatenate((np.eye(6)[testArray[:,0]],np.eye(2)[testArray[:,1]-1],np.eye(8)[testArray[:,2]],np.eye(8)[testArray[:,3]],testArray[:,4].reshape(testArray[:,4].size,1)),axis=1)
    int_prediction=regressor.predict(test_case)
    
    #Prediction with Strike rate and economy rate
    flt_prediction=regressor1.predict(rateArray)
    prediction=(flt_prediction+int_prediction)/2
    
    return int(prediction)





In [37]:
# %load main.py
### Imports ###
# add imports - classes and defs
from predictor import predictRuns


"""
sys.argv[1] is the input test file name given as command line arguments

"""
runs = predictRuns('May-02-inn2-match2.csv')
print("Predicted Runs: ", runs)

Predicted Runs:  53


In [10]:
runs
test_case
Mohammed Shami,A Singh,MC Henriques,Arshdeep Singh,DJ Hooda
D Padikkal,V Kohli,GJ Maxwell,RM Patidar,Washington Sundar


SyntaxError: invalid syntax (<ipython-input-10-f1788c443837>, line 3)

In [972]:
from predictor import predictRuns
predictRuns('inputFile.csv')

0

In [7]:
%cd C:\Users\user\Documents\Hackathon

C:\Users\user\Documents\Hackathon
